# 🏗️ Combined YOLOv8 Training - ALL Defects (Google Colab)

Train **ONE unified model** to detect:
- 🔴 **Cracks** - Structural cracks
- 🧱 **Spalling** - Concrete deterioration  
- 🔩 **Exposed Rebar** - Visible reinforcement bars

---

## ⚡ Quick Start

### Before You Begin:
1. **Enable GPU**: Runtime → Change runtime type → T4 GPU
2. **Get Roboflow API Key**: https://app.roboflow.com/settings/api
3. **Update crack dataset info** in Cell 6

### What This Does:
- Downloads **2 datasets** (crack + spalling/rebar)
- **Merges** into unified dataset
- **Trains single model** for all 3 classes
- **One model to detect everything!** 🎯

---

## 📋 Classes:
- **Class 0**: crack 🔴
- **Class 1**: spalling 🧱
- **Class 2**: exposed_rebar 🔩

Training time: ~3-4 hours on T4 GPU


---
## 1️⃣ Check GPU Availability


In [ ]:
import torch
import os

print("🔍 Checking GPU availability...\n")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU Memory: {gpu_memory:.2f} GB")
    device = '0'
    
    if gpu_memory < 15:
        print(f"\n⚠️  GPU has {gpu_memory:.1f}GB")
        print("💡 Use batch size 4")
    else:
        print(f"\n✅ GPU has {gpu_memory:.1f}GB")
        print("💡 Can use batch size 8")
else:
    print("\n⚠️ No GPU detected!")
    print("💡 Enable GPU: Runtime → Change runtime type → T4 GPU")
    device = 'cpu'

print(f"\n✅ Device: {device}")


---
## 2️⃣ Install Dependencies


In [ ]:
print("📦 Installing dependencies...\n")

!pip install ultralytics roboflow pyyaml -q

print("\n✅ All dependencies installed!")

import numpy as np
from ultralytics import YOLO
from roboflow import Roboflow
import yaml

print(f"\n📊 Versions:")
print(f"   NumPy: {np.__version__}")


---
## 3️⃣ Configure API Key


In [ ]:
# 🔑 YOUR ROBOFLOW API KEY
ROBOFLOW_API_KEY = "orloumjlWtpPXoxK5bFa"  # ✅ Configured

if ROBOFLOW_API_KEY == "YOUR_API_KEY_HERE":
    print("❌ ERROR: Replace with your API key!")
    raise ValueError("API key not configured")

print("✅ API key set")
print("🔑 Initializing Roboflow...")

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
print("✅ Roboflow ready")


---
## 4️⃣ Download Datasets

⚠️ **IMPORTANT**: Update CRACK_WORKSPACE and CRACK_PROJECT with your crack detection dataset info!


In [ ]:
print("📦 DOWNLOADING DATASETS\n" + "=" * 80)

# Dataset 1: Spalling & Exposed Rebar
print("\n📥 1/2: Downloading Spalling & Exposed Rebar...")
try:
    project1 = rf.workspace("labelling-9tvkx").project("spalling-and-exposed-rebar-ttsjj")
    dataset1 = project1.version(1).download("yolov8", location="/content/dataset_spalling")
    print(f"✅ Downloaded: {dataset1.location}")
    dataset1_location = dataset1.location
except Exception as e:
    print(f"❌ Failed: {e}")
    dataset1_location = None

# Dataset 2: Crack Detection  
print("\n📥 2/2: Downloading Crack Detection...")
print("⚠️  UPDATE THESE VALUES WITH YOUR CRACK DATASET INFO:")

# ⚠️ UPDATE THESE WITH YOUR ACTUAL CRACK DATASET INFO FROM ROBOFLOW
CRACK_WORKSPACE = "your-workspace-name"  # ← UPDATE THIS!
CRACK_PROJECT = "your-crack-project"      # ← UPDATE THIS!
CRACK_VERSION = 1                         # ← UPDATE IF NEEDED

try:
    project2 = rf.workspace(CRACK_WORKSPACE).project(CRACK_PROJECT)
    dataset2 = project2.version(CRACK_VERSION).download("yolov8", location="/content/dataset_crack")
    print(f"✅ Downloaded: {dataset2.location}")
    dataset2_location = dataset2.location
except Exception as e:
    print(f"❌ Failed: {e}")
    print("\n💡 To find your dataset info:")
    print("   1. Go to https://app.roboflow.com/")
    print("   2. Open your crack detection project")
    print("   3. Look at the URL: app.roboflow.com/WORKSPACE/PROJECT")
    print("   4. Update CRACK_WORKSPACE and CRACK_PROJECT above")
    print("\n⚠️  Continuing with spalling/rebar only...")
    dataset2_location = None

print("\n" + "=" * 80)


---
## 5️⃣ Merge Datasets

Combining both datasets with proper class remapping.


In [ ]:
from pathlib import Path
import shutil

print("🔄 MERGING DATASETS\n" + "=" * 80)

# Create combined dataset structure
combined_dir = Path("/content/combined_dataset")
combined_dir.mkdir(exist_ok=True)

for split in ['train', 'valid', 'test']:
    (combined_dir / split / 'images').mkdir(parents=True, exist_ok=True)
    (combined_dir / split / 'labels').mkdir(parents=True, exist_ok=True)

print("✅ Created structure\n")

# Class mapping:
# Dataset 1 (Spalling/Rebar): 0=spalling, 1=exposed_rebar
# Dataset 2 (Cracks): 0=crack
# Combined: 0=crack, 1=spalling, 2=exposed_rebar

def merge_dataset(src_path, prefix, class_remap):
    """Copy images and remap labels"""
    counts = {'train': 0, 'valid': 0, 'test': 0}
    
    for split in ['train', 'valid', 'test']:
        src_img = Path(src_path) / split / 'images'
        src_lbl = Path(src_path) / split / 'labels'
        
        if not src_img.exists():
            continue
        
        dst_img = combined_dir / split / 'images'
        dst_lbl = combined_dir / split / 'labels'
        
        # Copy each image
        for img_file in src_img.glob('*.*'):
            new_name = f"{prefix}_{img_file.name}"
            shutil.copy(img_file, dst_img / new_name)
            counts[split] += 1
            
            # Copy and remap label
            lbl_file = src_lbl / f"{img_file.stem}.txt"
            if lbl_file.exists():
                with open(lbl_file, 'r') as f:
                    lines = f.readlines()
                
                with open(dst_lbl / f"{prefix}_{img_file.stem}.txt", 'w') as f:
                    for line in lines:
                        parts = line.strip().split()
                        if parts:
                            old_id = int(parts[0])
                            new_id = class_remap.get(old_id, old_id)
                            parts[0] = str(new_id)
                            f.write(' '.join(parts) + '\n')
    
    return counts

# Merge dataset 1 (Spalling/Rebar)
if dataset1_location:
    print("📁 Merging Spalling & Rebar dataset...")
    # Remap: 0→1 (spalling), 1→2 (exposed_rebar)
    counts1 = merge_dataset(dataset1_location, 'sp', {0: 1, 1: 2})
    print(f"   Train: {counts1['train']}, Valid: {counts1['valid']}, Test: {counts1['test']}")
else:
    counts1 = {'train': 0, 'valid': 0, 'test': 0}
    print("⚠️  Spalling dataset not available")

# Merge dataset 2 (Cracks)
if dataset2_location:
    print("\n📁 Merging Crack dataset...")
    # Keep: 0→0 (crack)
    counts2 = merge_dataset(dataset2_location, 'cr', {0: 0})
    print(f"   Train: {counts2['train']}, Valid: {counts2['valid']}, Test: {counts2['test']}")
else:
    counts2 = {'train': 0, 'valid': 0, 'test': 0}
    print("\n⚠️  Crack dataset not available")

# Create data.yaml
data_config = {
    'path': str(combined_dir),
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'nc': 3,
    'names': ['crack', 'spalling', 'exposed_rebar']
}

with open(combined_dir / 'data.yaml', 'w') as f:
    yaml.dump(data_config, f)

print("\n" + "=" * 80)
print("✅ MERGING COMPLETE!")
print("=" * 80)
print(f"\n📊 Combined Dataset:")
print(f"   Train:  {counts1['train'] + counts2['train']} images")
print(f"   Valid:  {counts1['valid'] + counts2['valid']} images")
print(f"   Test:   {counts1['test'] + counts2['test']} images")
print(f"\n🎯 Classes:")
print(f"   0: crack 🔴")
print(f"   1: spalling 🧱")
print(f"   2: exposed_rebar 🔩")
print(f"\n📂 Location: {combined_dir}")


In [ ]:
CONFIG = {
    'model': 'yolov8n.pt',
    'data': str(combined_dir / 'data.yaml'),
    'epochs': 50,
    'imgsz': 512,
    'batch': 4,
    'device': device,
    'project': '/content/runs/detect',
    'name': 'combined_all_defects',
    'workers': 0,
    'cache': False,
    'patience': 10,
    'optimizer': 'SGD',
    'lr0': 0.01,
    'amp': True,
    'plots': True,
    'verbose': True,
}

print("⚙️  COMBINED MODEL CONFIGURATION")
print("=" * 80)
print("\n📊 Settings:")
print(f"   Model:      {CONFIG['model']}")
print(f"   Epochs:     {CONFIG['epochs']}")
print(f"   Image Size: {CONFIG['imgsz']}")
print(f"   Batch:      {CONFIG['batch']}")
print(f"   Device:     {CONFIG['device']}")
print(f"\n🎯 Detecting 3 classes:")
print(f"   0: crack 🔴")
print(f"   1: spalling 🧱")
print(f"   2: exposed_rebar 🔩")
print(f"\n📂 Output: {CONFIG['project']}/{CONFIG['name']}")
print("=" * 80)

if torch.cuda.is_available():
    est = CONFIG['epochs'] * 2
    print(f"\n⏰ Estimated: ~{est} min ({est/60:.1f} hours)")


---
## 7️⃣ Train Combined Model

Training ONE model for ALL 3 defect types!


In [ ]:
import time
import traceback
import gc

# Clear GPU
print("🧹 Clearing GPU...")
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    print("✅ GPU cleared\n")

print("\n" + "🚀" * 40)
print("TRAINING COMBINED MODEL - ALL DEFECTS")
print("🚀" * 40 + "\n")

try:
    print("🔧 Loading YOLOv8...")
    model = YOLO(CONFIG['model'])
    print("✅ Model loaded\n")
    
    if torch.cuda.is_available():
        mem = torch.cuda.memory_allocated(0) / 1024**3
        print(f"💾 GPU: {mem:.2f} GB\n")
    
    print("🏋️  Starting training...\n")
    print("=" * 80)
    print(f"📂 Results: {CONFIG['project']}/{CONFIG['name']}")
    print("=" * 80 + "\n")
    
    start_time = time.time()
    
    # Train
    results = model.train(**CONFIG)
    
    training_time = time.time() - start_time
    
    print("\n" + "=" * 80)
    print("🎉 TRAINING COMPLETED! 🎉")
    print("=" * 80)
    print(f"\n⏱️  Time: {training_time/3600:.2f} hours")
    print(f"📂 Model: {CONFIG['project']}/{CONFIG['name']}/weights/best.pt")
    print("\n✅ Your model now detects:")
    print("   • Cracks 🔴")
    print("   • Spalling 🧱")
    print("   • Exposed Rebar 🔩")
    print("\n💡 All in ONE model!")
    
except RuntimeError as e:
    print("\n❌ RUNTIME ERROR!")
    if "out of memory" in str(e).lower():
        print("🔥 GPU OOM!")
        print("\n💡 Reduce batch to 2 or imgsz to 416")
    traceback.print_exc()
    
except Exception as e:
    print(f"\n❌ ERROR: {type(e).__name__}")
    print(str(e))
    traceback.print_exc()
    
finally:
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("\n🧹 GPU cleared")


---
## 8️⃣ Save & Download Model


In [ ]:
from google.colab import files

results_dir = f"{CONFIG['project']}/{CONFIG['name']}"
best_model = f"{results_dir}/weights/best.pt"

if os.path.exists(best_model):
    # Copy to content root for easy access
    shutil.copy(best_model, "/content/combined_all_defects_best.pt")
    size = os.path.getsize('/content/combined_all_defects_best.pt') / 1024**2
    
    print("💾 COMBINED MODEL SAVED!")
    print("=" * 80)
    print("✅ File: /content/combined_all_defects_best.pt")
    print(f"📊 Size: {size:.2f} MB")
    print("\n🎯 Detects 3 classes:")
    print("   Class 0: crack 🔴")
    print("   Class 1: spalling 🧱")
    print("   Class 2: exposed_rebar 🔩")
    print("\n📥 Download the model:")
    print("=" * 80)
    
    # Download the model
    files.download('/content/combined_all_defects_best.pt')
    print("\n✅ Download started!")
    
    print("\n💻 Usage:")
    print("```python")
    print("from ultralytics import YOLO")
    print("model = YOLO('combined_all_defects_best.pt')")
    print("results = model('image.jpg')")
    print("results[0].show()  # Shows all 3 defect types!")
    print("```")
    print("=" * 80)
else:
    print("❌ Model not found")


---
## 9️⃣ Visualize Results


In [ ]:
from IPython.display import Image, display
import glob

results_dir = f"{CONFIG['project']}/{CONFIG['name']}"

print("📈 Training Results:\n")
print("=" * 80 + "\n")

# Results plot
if os.path.exists(f"{results_dir}/results.png"):
    print("📊 Training Curves:\n")
    display(Image(filename=f"{results_dir}/results.png", width=1000))

# Confusion matrix
if os.path.exists(f"{results_dir}/confusion_matrix.png"):
    print("\n🎯 Confusion Matrix:\n")
    display(Image(filename=f"{results_dir}/confusion_matrix.png", width=800))

# F1 curve
if os.path.exists(f"{results_dir}/F1_curve.png"):
    print("\n📈 F1-Confidence Curve:\n")
    display(Image(filename=f"{results_dir}/F1_curve.png", width=800))

# Validation predictions
val_images = sorted(glob.glob(f"{results_dir}/val_batch*_pred.jpg"))
if val_images:
    print("\n🔍 Validation Predictions:\n")
    for img in val_images[:2]:
        display(Image(filename=img, width=1000))


---
## 📝 Summary & Next Steps

### ✅ Training Complete!

You now have `combined_all_defects_best.pt` that detects:
- 🔴 **Crack** (class 0)
- 🧱 **Spalling** (class 1)
- 🔩 **Exposed Rebar** (class 2)

### 📥 Model Downloaded!

The model has been automatically downloaded to your computer.

### 🚀 Next Steps:

1. **Upload to your project**: Place `combined_all_defects_best.pt` in your demo folder
2. **Rename** (optional): Rename to `best.pt` to replace your current model
3. **Update webapp**: Your `finalwebapp.py` will now detect all 3 defect types!

### 💻 Quick Test:

```python
from ultralytics import YOLO

# Load your combined model
model = YOLO('combined_all_defects_best.pt')

# Test on an image
results = model('test_image.jpg')

# Show results
results[0].show()

# Get detections
for box in results[0].boxes:
    cls = int(box.cls[0])
    names = {0: 'crack', 1: 'spalling', 2: 'exposed_rebar'}
    conf = float(box.conf[0])
    print(f"{names[cls]}: {conf:.2%}")
```

### 🎯 Model Performance:

Check the visualizations above to see:
- Training curves (loss decreasing, mAP increasing)
- Confusion matrix (which classes it confuses)
- Validation predictions (actual detections)

---

**One model for everything! 🎯**

Replace your current `best.pt` with this combined model and your webapp will detect cracks, spalling, AND exposed rebar!
